# The Heart of Home — Gemini Pro 2.5 Batch Art Generator

This notebook generates 30 cohesive digital art scenes depicting seniors in housing communities.

## Prerequisites

1. Upload `the_heart_of_home_batch_full.json` to Colab
2. Have Vertex AI / Gemini Pro 2.5 API access enabled
3. Set your GCP Project ID below

## Setup

In [ ]:
# Configuration
PROJECT_ID = "YOUR_GCP_PROJECT_ID"  # ⚠️ CHANGE THIS
LOCATION = "us-central1"  # or your preferred region
BATCH_FILE = "the_heart_of_home_batch_full.json"
OUTPUT_BUCKET = "gs://your-output-bucket/heart_of_home_outputs"  # ⚠️ CHANGE THIS
LOCAL_SAVE_DIR = "heart_of_home_images"

## Install Dependencies

In [ ]:
!pip install -U google-cloud-aiplatform google-cloud-storage tqdm pillow

## Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print(f"✅ Authenticated and initialized for project: {PROJECT_ID}")

## Load Batch Prompts

In [ ]:
import json

with open(BATCH_FILE, "r") as f:
    batch_data = json.load(f)

print(f"✅ Loaded {len(batch_data['requests'])} prompts")
print(f"\nFirst prompt preview:")
print(f"ID: {batch_data['requests'][0]['custom_id']}")
print(f"Prompt: {batch_data['requests'][0]['prompt'][:100]}...")

## Generate Images via Gemini Pro 2.5

⚠️ **Note:** This cell will take several minutes to complete as it generates 30 images.

The Gemini API doesn't have native batch mode for image generation, so we'll submit requests sequentially with rate limiting.

In [ ]:
import os
import time
import base64
from tqdm import tqdm
from google.cloud import aiplatform
from vertexai.preview.vision_models import ImageGenerationModel

# Initialize the image generation model
model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")

# Create output directory
os.makedirs(LOCAL_SAVE_DIR, exist_ok=True)

print(f"🎨 Starting generation of {len(batch_data['requests'])} images...\n")

successful = 0
failed = 0

for idx, req in enumerate(tqdm(batch_data['requests'], desc="Generating images")):
    try:
        custom_id = req['custom_id']
        prompt = req['prompt']
        
        # Generate image
        images = model.generate_images(
            prompt=prompt,
            number_of_images=1,
            aspect_ratio="16:9",
            safety_filter_level="block_few",
            person_generation="allow_all"
        )
        
        # Save the image
        img_path = f"{LOCAL_SAVE_DIR}/{custom_id}.png"
        images[0].save(img_path)
        
        successful += 1
        
        # Rate limiting - wait between requests
        time.sleep(2)
        
    except Exception as e:
        print(f"\n❌ Failed for {req['custom_id']}: {str(e)}")
        failed += 1
        continue

print(f"\n✅ Generation complete!")
print(f"   Successful: {successful}")
print(f"   Failed: {failed}")
print(f"   Output directory: {LOCAL_SAVE_DIR}/")

## Preview Generated Images

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Show first 6 images as preview
image_files = sorted([f for f in os.listdir(LOCAL_SAVE_DIR) if f.endswith('.png')])[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_file in enumerate(image_files):
    img = Image.open(os.path.join(LOCAL_SAVE_DIR, img_file))
    axes[idx].imshow(img)
    axes[idx].set_title(img_file.replace('.png', '').replace('_', ' ').title(), fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(f"\nShowing preview of first 6 images out of {len(image_files)} total")

## Download Images as ZIP

In [ ]:
import shutil
from google.colab import files

# Create ZIP archive
zip_filename = "heart_of_home_images"
shutil.make_archive(zip_filename, 'zip', LOCAL_SAVE_DIR)

print(f"✅ Created {zip_filename}.zip")
print(f"\nDownloading to your computer...")

# Download the ZIP file
files.download(f"{zip_filename}.zip")

print(f"\n✅ Download complete!")

## Optional: Upload to Google Cloud Storage

In [ ]:
from google.cloud import storage

# Parse bucket name and path from OUTPUT_BUCKET
bucket_path = OUTPUT_BUCKET.replace('gs://', '')
bucket_name = bucket_path.split('/')[0]
blob_prefix = '/'.join(bucket_path.split('/')[1:]) if '/' in bucket_path else ''

# Initialize storage client
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name)

print(f"📤 Uploading to gs://{bucket_name}/{blob_prefix}")

uploaded = 0
for filename in tqdm(os.listdir(LOCAL_SAVE_DIR), desc="Uploading"):
    if filename.endswith('.png'):
        local_path = os.path.join(LOCAL_SAVE_DIR, filename)
        blob_path = f"{blob_prefix}/{filename}" if blob_prefix else filename
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(local_path)
        uploaded += 1

print(f"\n✅ Uploaded {uploaded} images to Cloud Storage")
print(f"   Location: {OUTPUT_BUCKET}")

## Create Metadata CSV

In [ ]:
import csv

# Create metadata CSV
metadata_file = "heart_of_home_metadata.csv"

with open(metadata_file, 'w', newline='') as csvfile:
    fieldnames = ['custom_id', 'filename', 'prompt', 'housing_type', 'scene_type']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for req in batch_data['requests']:
        custom_id = req['custom_id']
        
        # Extract scene type from custom_id (e.g., "balcony", "kitchen", etc.)
        scene_parts = custom_id.split('_')
        scene_type = '_'.join(scene_parts[1:-1]) if len(scene_parts) > 2 else custom_id
        
        # Determine housing type from prompt
        housing_type = "apartment"
        if "community" in req['prompt'].lower():
            housing_type = "community_space"
        elif "courtyard" in req['prompt'].lower():
            housing_type = "courtyard"
        
        writer.writerow({
            'custom_id': custom_id,
            'filename': f"{custom_id}.png",
            'prompt': req['prompt'],
            'housing_type': housing_type,
            'scene_type': scene_type
        })

print(f"✅ Created metadata file: {metadata_file}")
print(f"\nDownloading metadata CSV...")
files.download(metadata_file)

---

## 🎨 Best Practices for Consistency

### Style Lock
- All prompts use `"digital art, painterly realism, warm light"`
- Maintains visual cohesion across the entire collection

### Housing Continuity
- Consistent architectural elements (balconies, courtyards, community spaces)
- Unified color palette and lighting

### Post-Processing (Optional)
1. Apply a soft LUT for final color unification
2. Export web-safe PNGs at 2048×1152 or 1920×1080

### Integration Ideas
- Marketing carousel for senior realty app
- Onboarding collage showing housing diversity
- Generative cover art for blog/investor materials
- Website hero sections

---

**Generated with ❤️ for A Place of Your Own**